# Refactored BinaPs Demo

This notebook demonstrates the usability of an overhauled BinaPs implementation that is supposed to be more flexible and
developer friendly. It consists on three steps:

**1. Generate synthetic data:** With one of the provided scripts, we generate synthetic data in which a set of known
patterns are planted.

**2. Run BinaPs:** We'll run BinaPs over this synthetic dataset and get a list of inferred patterns

Copyright 2022 Bernardo C. Rodrigues

See COPYING file for license details

In [ ]:
import torch

# Check if CUDA is available
if(torch.cuda.is_available()):
    print(f"CUDA Device: {torch.cuda.get_device_name(0)}")
else:
    print("No CUDA device avaiable")

In [ ]:
import tempfile
from pathlib import Path
from binaps.binaps_wrapper import generate_synthetic_data

output_file = "data"

row_quantity = 1000
column_quantity = 10
max_pattern_size = 500
noise = 0.05
density = 0.03

tmp_dir = Path(tempfile.mkdtemp())

generate_synthetic_data(tmp_dir, row_quantity, column_quantity, output_file, max_pattern_size, noise, density)

In [ ]:
from torch import device, cuda

import numpy as np

from binaps.network import learn, get_patterns
from binaps.dataset import BinaryDataset, parse_dat_file, divide_data

data_file = "data.dat"
proportion = 0.9

device = device("cuda:0" if cuda.is_available() else "cpu")

data = parse_dat_file(data_file)
train_data, test_data = divide_data(data, 0.9)

train_dataset = BinaryDataset(train_data, device)
test_dataset = BinaryDataset(test_data, device)

batch_size = 64
test_batch_size = 64
hidden_dim = 20
lr = 0.01
weight_decay = 0
gamma = 0.1
epochs = 100

model, weights = learn(train_dataset, 
                       test_dataset,
                       batch_size,
                       test_batch_size,
                       hidden_dim,
                       lr,
                       weight_decay,
                       gamma,
                       epochs)


In [ ]:
patterns = get_patterns(weights)

for pattern in patterns:
    print(pattern)